## ViVa VCF Visualizer - Version 0.2


# These are the global variables you must define before running the notebook:
### Required options:  

#### `vcf_filename :: String` - This is a vector of filters that are applied to rows from the VCF file for visualization

#### `field_to_visualize :: String` - This is a vector of filters that are applied to rows from the VCF file for visualization

#### `variant_filter :: Vector{String}` - This is a vector of filters that are applied to rows from the VCF file for visualization
#### e.g. 
`variant_filter = ["pass_only", "chr1:20000000-30000000"]`

Variant_filter input options:
- `"all"` = no filter, visualize all variants
- `"pass_only"` = filter to select variant rows that pass samtools QC
- `"list.txt"` = select variants that match a user produced list of variant chromosome positions
- `"chr1:20000000-30000000"` = select variants within a genomic range in format "chr1:20000000-30000000"

Sample_filter input options:
- `"select_columns"` also requires -> `"sample_id_list.txt"`
- `"reorder_columns"` also requires -> `"phenotype_matrix.csv"` & `"phenotype_label_to_sort_by"`

#### `sample_filter :: Vector{String}` - This is a vector of filters that are applied to rows from the VCF file for visualization



In [7]:
vcf_filename = "variants.filtered.191_joint.vcf"
field_to_visualize = "read_depth" 
variant_filter = "all"
sample_filter = "select_columns", "select_column_list.txt"
save_format = "png"
plot_title = "Example_1"

"Example_1"

In [3]:
vcf_filename = "variants.filtered.191_joint.vcf"
field_to_visualize = "genotype" # field(s) = "read_depth", *etc.
variant_filter = "pass_only", "list", "significantList_for_proteinstructures.csv"#"range", "chr1:20000000-30000000"
sample_filter = "reorder_columns", "sample_phenotype_matrix.csv", "case_control_status" # "select_columns", "select_column_list.txt",
save_format = "pdf"
plot_title = "Example_2"




"Example_2"

### Notes to self
1) define vcf in begin to save as original version for second analysis
2) save notebook as script
3) CSV.read.jl
4) replace flag arguments (-a, -r, etc. with all, range, list, etc. and fix in notebook example inputs)
5) turn interactive display plotlyJS off 
6) turn extraneous code to functions
7) set default values - e.g. heatmap save to pdf by default
8) set up vector
9) categorical heatmap in PlotlyJS 




In [15]:
#=
using DataFrames #use CSV.jl ? depwarnings
using CSV
using PlotlyJS
using Rsvg
using Blink 
=#

using ViVa

ViVa.jupyter_main(vcf_filename,field_to_visualize,variant_filter,sample_filter,save_format,plot_title)


24146×200 DataFrames.DataFrame. Omitted printing of 194 columns
│ Row   │ #CHROM │ POS       │ ID  │ REF │ ALT │ QUAL      │
├───────┼────────┼───────────┼─────┼─────┼─────┼───────────┤
│ 1     │ 1      │ 11994687  │ 0.0 │ T   │ C   │ 566.43    │
│ 2     │ 1      │ 11994701  │ 0.0 │ G   │ A   │ 177.19    │
│ 3     │ 1      │ 11994760  │ 0.0 │ G   │ A   │ 891.27    │
│ 4     │ 1      │ 11994929  │ 0.0 │ C   │ T   │ 414.53    │
│ 5     │ 1      │ 11995005  │ 0.0 │ T   │ C   │ 130.93    │
│ 6     │ 1      │ 12008203  │ 0.0 │ C   │ T   │ 18454.9   │
│ 7     │ 1      │ 12009729  │ 0.0 │ G   │ A   │ 1070.08   │
│ 8     │ 1      │ 12009731  │ 0.0 │ C   │ A   │ 700.62    │
⋮
│ 24138 │ X      │ 153282564 │ 0.0 │ CT  │ C   │ 1398.78   │
│ 24139 │ X      │ 153282595 │ 0.0 │ G   │ A   │ 88.58     │
│ 24140 │ X      │ 153283434 │ 0.0 │ C   │ T   │ 3689.36   │
│ 24141 │ X      │ 153284172 │ 0.0 │ A   │ T   │ 12716.8   │
│ 24142 │ X      │ 153284192 │ 0.0 │ A   │ G   │ 7.31766e5 │
│ 24143 │ X      │ 

In [16]:
readvcf = readlines(vcf_filename)

for row = 1:size(readvcf,1)

      if contains(readvcf[row], "#CHROM")
          header_col = row
          global header_col
          header_string = readvcf[row]
      end
end

df_withsamplenames = CSV.read(vcf_filename, delim="\t", datarow = header_col+1, header = false, types=Dict(1=>String))
Base.promote_rule(::Type{C}, ::Type{Any}) where {C <: CategoricalArrays.CatValue} = Any
matr=Matrix(df_withsamplenames)

#vcf_csv=CSV.read(vcf_filename, delim="\t", datarow = header_col+1, header = header_col, types=Dict(1=>String))

24146×200 Array{Any,2}:
 "1"   11994687  0.0  "T"   "C"  …  "0/0:24,0:24:63:0,63,945"        
 "1"   11994701  0.0  "G"   "A"     "0/0:27,0:27:81:0,81,958"        
 "1"   11994760  0.0  "G"   "A"     "0/0:35,0:35:99:0,102,1399"      
 "1"   11994929  0.0  "C"   "T"     "0/0:33,0:33:90:0,90,1350"       
 "1"   11995005  0.0  "T"   "C"     "0/0:11,0:11:33:0,33,401"        
 "1"   12008203  0.0  "C"   "T"  …  "0/0:251,0:251:99:0,120,1800"    
 "1"   12009729  0.0  "G"   "A"     "0/0:85,0:85:99:.:.:0,120,1800"  
 "1"   12009731  0.0  "C"   "A"     "0/0:85,0:85:99:.:.:0,120,1800"  
 "1"   12009734  0.0  "C"   "T"     "0/1:57,34:91:99:843,0,1746"     
 "1"   12009749  0.0  "T"   "A"     "0/0:94,0:94:99:.:.:0,120,1800"  
 "1"   12009753  0.0  "C"   "A"  …  "0/0:94,0:94:99:.:.:0,120,1800"  
 "1"   12009754  0.0  "A"   "G"     "0/0:94,0:94:99:.:.:0,120,1800"  
 "1"   12009757  0.0  "A"   "G"     "0/0:94,0:94:99:.:.:0,120,1800"  
 ⋮                               ⋱                                

In [13]:
matr

24146×200 Array{Any,2}:
 "1"   11994687  0.0  "T"   "C"  …  "0/0:24,0:24:63:0,63,945"        
 "1"   11994701  0.0  "G"   "A"     "0/0:27,0:27:81:0,81,958"        
 "1"   11994760  0.0  "G"   "A"     "0/0:35,0:35:99:0,102,1399"      
 "1"   11994929  0.0  "C"   "T"     "0/0:33,0:33:90:0,90,1350"       
 "1"   11995005  0.0  "T"   "C"     "0/0:11,0:11:33:0,33,401"        
 "1"   12008203  0.0  "C"   "T"  …  "0/0:251,0:251:99:0,120,1800"    
 "1"   12009729  0.0  "G"   "A"     "0/0:85,0:85:99:.:.:0,120,1800"  
 "1"   12009731  0.0  "C"   "A"     "0/0:85,0:85:99:.:.:0,120,1800"  
 "1"   12009734  0.0  "C"   "T"     "0/1:57,34:91:99:843,0,1746"     
 "1"   12009749  0.0  "T"   "A"     "0/0:94,0:94:99:.:.:0,120,1800"  
 "1"   12009753  0.0  "C"   "A"  …  "0/0:94,0:94:99:.:.:0,120,1800"  
 "1"   12009754  0.0  "A"   "G"     "0/0:94,0:94:99:.:.:0,120,1800"  
 "1"   12009757  0.0  "A"   "G"     "0/0:94,0:94:99:.:.:0,120,1800"  
 ⋮                               ⋱                                